In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import sys
from pathlib import Path
import re
import os

import blpapi
import clarion
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid
from pymongo import MongoClient

from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [5]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

In [6]:
client = MongoClient(
    host='localhost',
    port=27017
)

In [ ]:
db = client.test

In [7]:
curve_hist

date         security  last_price tenor  \
0      2009-03-31  USSWAP10 Curncy      2.9270   10Y   
1      2009-04-01  USSWAP10 Curncy      2.8750   10Y   
2      2009-04-02  USSWAP10 Curncy      2.8680   10Y   
3      2009-04-03  USSWAP10 Curncy      2.9450   10Y   
4      2009-04-06  USSWAP10 Curncy      3.0788   10Y   
...           ...              ...         ...   ...   
186659 2022-06-22   USSWAP2 Curncy      3.4539    2Y   
186660 2022-06-23   USSWAP2 Curncy      3.3784    2Y   
186661 2022-06-24   USSWAP2 Curncy      3.4039    2Y   
186662 2022-06-27   USSWAP2 Curncy      3.4389    2Y   
186663 2022-06-28   USSWAP2 Curncy      3.4523    2Y   

                                curve_name  
0       USD ISDA Standard Rates Swap Curve  
1       USD ISDA Standard Rates Swap Curve  
2       USD ISDA Standard Rates Swap Curve  
3       USD ISDA Standard Rates Swap Curve  
4       USD ISDA Standard Rates Swap Curve  
...                                    ...  
186659      USD ISDA CDS Fixing SWAP CURVE  
186660      USD ISDA CDS Fixing SWAP CURVE  
186661      USD ISDA CDS Fixing SWAP CURVE  
186662      USD ISDA CDS Fixing SWAP CURVE  
186663      USD ISDA CDS Fixing SWAP CURVE  

[186664 rows x 5 columns]